In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

np.set_printoptions(linewidth=np.inf)
np.set_printoptions(edgeitems=30) 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# MLflow
import mlflow
import mlflow.tensorflow

In [ ]:
# 2. Configuration MLflow Simple
def setup_mlflow():
    """Configuration MLflow basique"""
    
    # Configurer l'URI local
    mlflow.set_tracking_uri("file:./mlruns")
    
    # Nom de l'expérience
    experiment_name = "pneumonia_test"
    
    # Créer ou récupérer l'expérience
    try:
        experiment_id = mlflow.create_experiment(experiment_name)
        print(f"✅ Expérience '{experiment_name}' créée")
    except:
        print(f"✅ Expérience '{experiment_name}' existe déjà")
    
    # Activer l'expérience
    mlflow.set_experiment(experiment_name)
    print("🎯 MLflow configuré")
    # Configurer MLflow
    setup_mlflow()

In [4]:
# 3. Test MLflow basique
def test_mlflow():
    """Test simple de MLflow"""
    
    print("🧪 Test MLflow...")
    
    with mlflow.start_run(run_name="test_basique"):
        # Logger quelques paramètres de test
        mlflow.log_param("test_param", "ok")
        mlflow.log_param("python_version", "3.10")
        mlflow.log_param("tensorflow_version", tf.__version__)
        
        # Logger quelques métriques de test
        mlflow.log_metric("test_metric", 0.95)
        mlflow.log_metric("accuracy", 0.85)
        
        print("📝 Paramètres et métriques loggés")
    
    print("✅ Test MLflow réussi")
    # Tester MLflow
    test_mlflow()

## Exlploration des donées 

In [ ]:
# 4. Vérification structure dataset
def check_dataset_structure():
    """Vérifier si le dataset est présent et bien structuré"""
    
    print("🔍 Vérification du dataset...")
    
    base_path = "data/chest_xray"
    
    if not os.path.exists(base_path):
        print(f"❌ Dataset non trouvé dans {base_path}")
        print("💡 Téléchargez le dataset Chest X-Ray Pneumonia de Kaggle")
        return False
    
    # Vérifier la structure
    splits = ['train', 'test', 'val']
    classes = ['NORMAL', 'PNEUMONIA']
    
    structure_ok = True
    
    for split in splits:
        split_path = os.path.join(base_path, split)
        if os.path.exists(split_path):
            print(f"✅ {split}/ trouvé")
            
            for class_name in classes:
                class_path = os.path.join(split_path, class_name)
                if os.path.exists(class_path):
                    count = len([f for f in os.listdir(class_path) 
                               if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                    print(f"  {class_name}: {count} images")
                else:
                    print(f"  ❌ {class_name}/ manquant")
                    structure_ok = False
        else:
            print(f"❌ {split}/ manquant")
            structure_ok = False
    
    if structure_ok:
        print("✅ Structure du dataset OK")
    else:
        print("❌ Structure du dataset incomplète")
    
    return structure_ok

# Vérifier le dataset
dataset_ok = check_dataset_structure()

# 5. Exploration rapide (si dataset disponible)
def quick_exploration():
    """Exploration rapide du dataset"""
    
    if not dataset_ok:
        print("⚠️ Dataset non disponible - sautant l'exploration")
        return
    
    print("🔍 Exploration rapide...")

## Chargement des images 

In [ ]:
 # Charger quelques images d'exemple
base_path = "data/chest_xray/train"
    
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
for i, class_name in enumerate(['NORMAL', 'PNEUMONIA']):
    class_path = os.path.join(base_path, class_name)
        
    if os.path.exists(class_path):
            # Prendre la première image
            files = [f for f in os.listdir(class_path) 
                    if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            if files:
                img_path = os.path.join(class_path, files[0])
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                
                axes[i].imshow(img, cmap='gray')
                axes[i].set_title(f'Exemple {class_name}')
                axes[i].axis('off')
                
                print(f"📊 {class_name}: shape {img.shape}, type {img.dtype}")
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Exploration terminée")


In [ ]:
    # Explorer si possible 
quick_exploration()

In [ ]:
# 6. Chargement simplifié d'un échantillon
def load_sample_data(n_samples=10):
    """Charger un petit échantillon pour tester"""
    
    if not dataset_ok:
        print("⚠️ Création de données factices pour test")
        # Données factices pour tester le code
        X = np.random.rand(20, 64, 64, 1)
        y = np.random.randint(0, 2, 20)
        return X, y
    
    print(f"📂 Chargement de {n_samples} échantillons par classe...")
    
    X = []
    y = []
    
    base_path = "data/chest_xray/train"
    label_map = {'NORMAL': 0, 'PNEUMONIA': 1}
    
    for class_name in ['NORMAL', 'PNEUMONIA']:
        class_path = os.path.join(base_path, class_name)
        files = [f for f in os.listdir(class_path) 
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        # Prendre seulement n_samples
        for i, filename in enumerate(files[:n_samples]):
            img_path = os.path.join(class_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            if img is not None:
                # Redimensionner petit pour le test
                img = cv2.resize(img, (64, 64))
                img = img.reshape(64, 64, 1)  # Ajouter dimension channel
                
                X.append(img)
                y.append(label_map[class_name])
    
    X = np.array(X, dtype=np.float32) / 255.0  # Normaliser
    y = np.array(y)
    
    print(f"✅ Échantillon chargé: {X.shape[0]} images, shape {X.shape[1:]}")
    print(f"📊 Distribution: {np.bincount(y)}")
    
    return X, y

# Charger un échantillon
X_sample, y_sample = load_sample_data(n_samples=10)


In [ ]:
# 7. Modèle CNN simple pour test
def create_simple_model():
    """Créer un CNN très simple pour test"""
    
    model = models.Sequential([
        layers.Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    print("✅ Modèle simple créé")
    print(f"📊 Paramètres: {model.count_params():,}")
    
    return model

# Créer le modèle
model = create_simple_model()

In [ ]:
#  8. Test d'entraînement avec MLflow
def test_training_with_mlflow():
    """Test d'entraînement court avec MLflow"""
    
    print("🏋️ Test d'entraînement avec MLflow...")
    
    with mlflow.start_run(run_name="test_entrainement"):
        
        # Logger les paramètres
        mlflow.log_param("model_type", "CNN_simple")
        mlflow.log_param("epochs", 3)
        mlflow.log_param("batch_size", 8)
        mlflow.log_param("samples", len(X_sample))
        
        # Entraînement très court
        history = model.fit(
            X_sample, y_sample,
            epochs=3,
            batch_size=8,
            verbose=1
        )
        
        # Logger les métriques finales
        final_loss = history.history['loss'][-1]
        final_accuracy = history.history['accuracy'][-1]
        
        mlflow.log_metric("final_loss", final_loss)
        mlflow.log_metric("final_accuracy", final_accuracy)
        
        # Sauvegarder le modèle
        mlflow.tensorflow.log_model(model, "simple_model")
        
        print(f"✅ Entraînement terminé")
        print(f"📊 Loss finale: {final_loss:.3f}")
        print(f"📊 Accuracy finale: {final_accuracy:.3f}")

# Tester l'entraînement
test_training_with_mlflow()

In [ ]:
# %% 9. Vérification finale
def final_check():
    """Vérification que tout fonctionne"""
    
    print("\n🔍 Vérification finale...")
    
    # Vérifier MLflow
    if os.path.exists("mlruns"):
        print("✅ Dossier mlruns créé")
    else:
        print("❌ Problème avec mlruns")
    
    # Vérifier le modèle
    try:
        predictions = model.predict(X_sample[:2])
        print(f"✅ Modèle fonctionnel - prédictions: {predictions.flatten()}")
    except Exception as e:
        print(f"❌ Problème modèle: {e}")
    
    print("\n🎉 Tests terminés!")
    print("\n💡 Prochaines étapes:")
    print("1. Lancez 'mlflow ui' dans le terminal")
    print("2. Allez sur http://localhost:5000")
    print("3. Vérifiez vos expériences 'pneumonia_test'")
    print("4. Si tout fonctionne, passez à la version complète!")

# Vérification finale
final_check()